# Preparing edge table

## Summary

- Import node table from Natalie.
- Extract list of traits.
- Identify sources of GWAS for the traits.
- Save table with links to test, specifying whether a cis-analysis or a genome-wide analysis is needed.

## Libraries

In [2]:
library(readr)
library(dplyr)
library(tidyr)
library(purrr)
library(ggplot2)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [3]:
projfold <- "/Users/da1078co/Documents/Lund/PhD/Projects/BN"

## Edges to test - from Natalie

In [4]:
edge_tb0 <- read_tsv(
  file.path(projfold, "data", "edge_table_original.tsv"),
  show_col_types = FALSE
)

In [5]:
head(edge_tb0)

node1,node2
<chr>,<chr>
AGRP,CD4
AGRP,Glucose
AGRP,TG
APOM,Clins
APOM,HDL
APOM,TG


## List of traits

In [6]:
trait_tb0 <- tibble(
  trait = c(
    edge_tb0[, 1, drop = TRUE],
    edge_tb0[, 2, drop = TRUE]
  )
) |>
  unique() |>
  arrange(trait)

In [7]:
trait_tb0$trait

[1] "AGRP"         "APOM"         "BasalISR"     "CD4"          "CDH5"        
 [6] "CTRC"         "CTSD"         "Clins"        "Clinsb"       "FAM3C"       
[11] "FGF21"        "Glucagonmin0" "Glucose"      "GlucoseSens"  "HDL"         
[16] "HMOX1"        "HOMA_IR"      "HbA1c"        "IGFBP1"       "IGFBP2"      
[21] "Insulin"      "KITLG"        "LDLR"         "LEP"          "LPL"         
[26] "LiverFat"     "MATN2"        "MFGE8"        "NRP1"         "OGIS"        
[31] "PON3"         "PancFat"      "SAT"          "TG"           "TGM2"        
[36] "TotGLP1min0"  "TwoGlucose"   "TwoInsulin"   "VAT"

## Gene traits

In [8]:
gene_traits <- c(
  "AGRP", "APOM", "CD4", "CDH5", "CTRC", "CTSD", "FAM3C",
  "FGF21", "HMOX1", "IGFBP1", "IGFBP2", "KITLG", "LDLR",
  "LEP", "LPL", "MATN2", "MFGE8", "NRP1", "PON3", "TGM2"
)

### GWAS source

We will use UKB-PPP summary statistics for these genes.

The process to download the files corresponding to these genes is described in:

- `03a_Download_pQTLPanel.py`
- `03b_Process_pQTLPanel.ipynb`
- `03c_Download_pQTL.ipynb`

### Gencode table to obtain gene coordinates

In [9]:
gencode <- read_tsv(
  file.path(
    "/Users/da1078co/Documents/Data/GENCODE/",
    "gencode.v47lift37.basic.annotation.gtf.gz"
  ),
  comment = "#",
  col_types = "cccnn-c-c",
  col_names = c(
    "CHROM", "source", "gene_type",
    "start", "end", "strand", "addinfo"
  )
) |>
  filter(gene_type == "gene") |>
  select(-gene_type) |>
  mutate(
    CHROM = gsub("chr", "", CHROM),
    gene_id = gsub(
      "^gene_id \"([^\"]+)\";.*",
      "\\1",
      addinfo
    ),
    gene_type = gsub(
      ".*gene_type \"([^\"]+)\";.*",
      "\\1",
      addinfo
    ),
    gene_name = gsub(
      ".*gene_name \"([^\"]+)\";.*",
      "\\1",
      addinfo
    )
  ) |>
  select(-addinfo)
head(gencode)

CHROM,source,start,end,strand,gene_id,gene_type,gene_name
<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
1,HAVANA,10370,13118,+,ENSG00000308415.1_1,lncRNA,DDX11L2
1,HAVANA,11121,24894,+,ENSG00000290825.2_2,lncRNA,DDX11L16
1,HAVANA,12010,13670,+,ENSG00000223972.6_6,transcribed_unprocessed_pseudogene,DDX11L1
1,HAVANA,14356,30744,-,ENSG00000310526.1_1,lncRNA,WASH7P
1,HAVANA,14696,24886,-,ENSG00000227232.6_7,transcribed_unprocessed_pseudogene,WASH7P
1,HAVANA,28589,31109,+,ENSG00000243485.6_13,lncRNA,MIR1302-2HG


### Adding cis regions

In [10]:
trait_tbg <- trait_tb0 |>
  filter(trait %in% gene_traits) |>
  inner_join(gencode, by = c("trait" = "gene_name")) |>
  select(trait, CHROM, start, end)

In [11]:
trait_tbg

trait,CHROM,start,end
<chr>,<chr>,<dbl>,<dbl>
AGRP,16,67516474,67517450
APOM,6,31620193,31625987
CD4,12,6896024,6929965
CDH5,16,66400593,66438687
CTRC,1,15764938,15775737
CTSD,11,1773982,1785803
FAM3C,7,120988932,121036418
FGF21,19,49258781,49261590
HMOX1,22,35776354,35790207


## Other traits

In [12]:
trait_tbng <- trait_tb0 |>
  filter(!trait %in% gene_traits)
trait_tbng$trait

[1] "BasalISR"     "Clins"        "Clinsb"       "Glucagonmin0" "Glucose"     
 [6] "GlucoseSens"  "HDL"          "HOMA_IR"      "HbA1c"        "Insulin"     
[11] "LiverFat"     "OGIS"         "PancFat"      "SAT"          "TG"          
[16] "TotGLP1min0"  "TwoGlucose"   "TwoInsulin"   "VAT"

### GWAS sources

| Acronym      | Trait                            | GWAS source                                               |
|--------------|----------------------------------|-----------------------------------------------------------|
| BasalISR     | Basal insulin secretion rate     | IVGTT GWAS from Wood et al 2017                           |
| Clins        | Insulin clearance                | Not found                                                 |
| Clinsb       | Basal insulin clearance          | Not found                                                 |
| Glucagonmin0 | Fasting glucagon                 | Not found                                                 |
| Glucose      | Fasting glucose                  | MAGIC consortium - Chen et al 2021                        |
| GlucoseSens  | Glucose sensitivity              | xinsdG30 from Madsen et al 2024                           |
| HDL          | High-density lipoprotein         | HDL from GLGC - Graham et al 2021                         |
| HOMA_IR      | Insulin resistance index         | MAGIC consortium - Dupuis et al 2011                      |
| HbA1c        | Glycated hemoglobin              | MAGIC consortium - Chen et al 2021                        |
| Insulin      | Fasting insulin                  | MAGIC consortium - Chen et al 2021                        |
| LiverFat     | Liver fat content                | Liu et al 2021                                            |
| OGIS         | Oral glucose insulin sensitivity | Not found                                                 |
| PancFat      | Pancreatic fat content           | Liu et al 2021                                            |
| SAT          | Subcutaneous adipose tissue      | Liu et al 2021                                            |
| TG           | Triglycerides                    | GLGC consortium - Graham et al 2021                       |
| TotGLP1min0  | Fasting GLP-1 levels             | Not found                                                 |
| TwoGlucose   | OGTT two-hour glucose            | MAGIC consortium - Chen et al 2021                        |
| TwoInsulin   | OGTT two-hour insulin            | MAGIC consortium - Chen et al 2021                        |
| VAT          | Visceral adipose tissue          | Liu et al 2021                                            |

Based on this table, only including traits for which we found GWAS summary statistics:

In [35]:
tormv <- c(
  "Clins", "Clinsb", "Glucagonmin0", "OGIS", "TotGLP1min0"
)

In [36]:
trait_tbngf <- trait_tbng |>
  filter(!trait %in% tormv)

URLs used and download process are described in `03_Download_GWAS.sh`

## New table with edges to test

In [43]:
edge_tbf <- bind_rows(
  # Testing links in both directions
  tibble(
    node1 = edge_tb0[, 1, drop = TRUE],
    node2 = edge_tb0[, 2, drop = TRUE]
  ),
  tibble(
    node1 = edge_tb0[, 2, drop = TRUE],
    node2 = edge_tb0[, 1, drop = TRUE]
  )
) |>
  # Only nodes for which we have GWAS summary statistics
  filter(
    !node1 %in% tormv,
    !node2 %in% tormv
  ) |>
  # Specify type of analysis that should be carried out
  mutate(
    atype = ifelse(node1 %in% gene_traits, "SMRHEIDI", "IVWMR")
  ) |>
  # Adding chromosome and position information for gene traits
  left_join(trait_tbg, by = c("node1" = "trait"))

In [44]:
edge_tbf

node1,node2,atype,CHROM,start,end
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
AGRP,CD4,SMRHEIDI,16,67516474,67517450
AGRP,Glucose,SMRHEIDI,16,67516474,67517450
AGRP,TG,SMRHEIDI,16,67516474,67517450
APOM,HDL,SMRHEIDI,6,31620193,31625987
APOM,TG,SMRHEIDI,6,31620193,31625987
BasalISR,GlucoseSens,IVWMR,NA,NA,NA
BasalISR,HDL,IVWMR,NA,NA,NA
BasalISR,HOMA_IR,IVWMR,NA,NA,NA
BasalISR,IGFBP1,IVWMR,NA,NA,NA


## Saving

In [45]:
write_tsv(
  edge_tbf,
  file.path(projfold, "data", "edge_table_totest.tsv")
)